In [1]:
import pandas as pd
import numpy as np
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import spearmanr as scor

/Users/benceszalai/Library/Enthought/Canopy/edm/envs/DeepLearningStart/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## At first we should get cell viabilities from CTRP data.

In [2]:
# read raw CTRP cell viability data
ctrp_raw=pd.read_table('../data/CTRP/v20.data.per_cpd_post_qc.txt',
                       sep='\t',header=0,index_col=None)
#read CTRP metadata
cell_info=pd.read_table('../data/CTRP/v20.meta.per_cell_line.txt',
                        sep='\t',header=0,index_col=None)
compound_info=pd.read_table('../data/CTRP/v20.meta.per_compound.txt',
                            sep='\t',header=0,index_col=None)
experiment_info=pd.read_table('../data/CTRP/v20.meta.per_experiment.txt',
                              sep='\t',header=0,index_col=None)

In [54]:
#using information from metadata we prepocess CTRP data
#to have usable format to match with LINCS
ctrp_proc=ctrp_raw.loc[:,['experiment_id','master_cpd_id',
                          'cpd_conc_umol','cpd_avg_pv']].copy()
print('Preprocessing CTRP',end='')
experiment_info=experiment_info.drop_duplicates(['experiment_id',
                                                 'master_ccl_id'])
print('.',end='')
experiment_info.index=experiment_info['experiment_id']
print('.',end='')
ctrp_proc['master_ccl_id']=experiment_info.loc[ctrp_proc['experiment_id'].values,
                                               'master_ccl_id'].values
print('.',end='')
cell_info.index=cell_info['master_ccl_id']
print('.',end='')
ctrp_proc['ccl_name']=cell_info.loc[ctrp_proc['master_ccl_id'].values,
                                    'ccl_name'].values
print('.',end='')
compound_info.index=compound_info['master_cpd_id']
print('.',end='')
ctrp_proc['broad_cpd_id']=compound_info.loc[ctrp_proc['master_cpd_id'].values,
                                            'broad_cpd_id'].values
print('.')
ctrp_proc=ctrp_proc.loc[:,['ccl_name','broad_cpd_id',
                           'cpd_conc_umol','cpd_avg_pv']]
print('Done!')

Preprocessing CTRP.......
Done!


Now we have a dataframe with Cell lines, Compound IDs, Compound concentrations and Cell viabilities as columns

In [55]:
print(ctrp_proc.head())
print('Shape:',ctrp_proc.shape)

  ccl_name   broad_cpd_id  cpd_conc_umol  cpd_avg_pv
0     CAS1  BRD-K46556387        0.00030      0.9303
1     CAS1  BRD-K46556387        0.00061      0.8337
2     CAS1  BRD-K46556387        0.00120      1.0460
3     CAS1  BRD-K46556387        0.00240      1.0910
4     CAS1  BRD-K46556387        0.00490      1.0190
Shape: (6171005, 4)


Now we can read some  metadata, to select the intersection (same cell line, compound) between LINCS-L1000 and CTRP.

In [56]:
#cell line metadata is same for gse92742 and gse70138
gse92742_cell=pd.read_table('../data/LINCS/GSE70138/GSE92742_Broad_LINCS_cell_info.txt',
                            sep='\t',header=0,index_col=None) 
lincs_cells=list(set(gse92742_cell['cell_id']))
gse92742_comp=pd.read_table('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_pert_info.txt',
                            sep='\t',header=0,index_col=None)
gse70138_comp=pd.read_table('../data/LINCS/GSE70138/GSE70138_Broad_LINCS_pert_info.txt',
                            sep='\t',header=0,index_col=None)
lincs_compounds=list(set(gse92742_comp['pert_id'])|set(gse70138_comp['pert_id']))

In [57]:
#we select data from ctrp with lincs cell lines and compounds
fil=np.in1d(ctrp_proc['ccl_name'],lincs_cells)&np.in1d(ctrp_proc['broad_cpd_id'],lincs_compounds)
ctrp_proc=ctrp_proc[fil]
print('Shape:',ctrp_proc.shape)

Shape: (238005, 4)


For duplicated entries in CTRP (same cell line, same drug, same concentraion) we calculate and keep average cell viability (of course, being duplicated for floats like concentrations is a bit complicated, but here we want only to remove real duplicated elements). 

In [58]:
ctrp_proc=ctrp_proc.groupby(['ccl_name','broad_cpd_id','cpd_conc_umol']).mean()
ctrp_proc.reset_index(inplace=True)
print('Shape:',ctrp_proc.shape)
ctrp_proc.to_csv('../results/CTRP/cell_viability_ctrp.csv',sep=',')

Shape: (221306, 4)


## Now we can match CTRP and LINCS-L1000
We will match CTRP and LINCS-L1000 instances based on cell line, drug and concentration. As the concentrations in CTRP and LINCS-L1000 are not the same, we will accept match LINCS-L1000 with the CTRP instance with the closest concentration (and same cell line and drug), as far as the absolute log10 concentration difference is smaller than 0.2 (~1.5 fold concetration difference).

In [59]:
def get_closest_cc_ctrp(l1000):
    """Selects the closest concentration instance from CTRP
    for a given L1000 instance, uses ctrp global variable CTRP"""
    fil=(CTRP['broad_cpd_id']==l1000['pert_id']) & (CTRP['ccl_name']==l1000['cell_id'])
    if np.sum(fil)>0:
        ctrp_temp=CTRP[fil].copy()
        ctrp_temp['delta_cc']=np.abs(ctrp_temp['log10_cpd_conc_umol']-l1000['log10_pert_dose'])
        j=ctrp_temp.sort_values('delta_cc').index[0]
        return ctrp_temp.loc[j,['cpd_avg_pv','log10_cpd_conc_umol']]
    else:
        return np.nan

In [60]:
# just read data
CTRP=pd.read_table('../results/CTRP/cell_viability_ctrp.csv',sep=',',
                       header=0,index_col=[0])
CTRP['log10_cpd_conc_umol']=np.log10(CTRP['cpd_conc_umol'])
cell_lines=list(set(CTRP['ccl_name']))
compounds=list(set(CTRP['broad_cpd_id']))
#read lincs metadata
sig_info_gse92742=pd.read_table('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_sig_info.txt',
                                sep='\t',header=0,index_col=None,low_memory=False)
sig_info_gse70138=pd.read_table('../data/LINCS/GSE70138/GSE70138_Broad_LINCS_sig_info.txt',
                                sep='\t',header=0,index_col=None,low_memory=False)
print('Shape GSE92742 sig_info:',sig_info_gse92742.shape)
print('Shape GSE70138 sig_info:',sig_info_gse70138.shape)

Shape GSE92742 sig_info: (473647, 12)
Shape GSE70138 sig_info: (118050, 8)


In [61]:
#filter for cell line and compound intersection with ctrp
fil=np.in1d(sig_info_gse92742['pert_id'],compounds) & np.in1d(sig_info_gse92742['cell_id'],cell_lines)
sig_info_gse92742=sig_info_gse92742[fil]    
fil=np.in1d(sig_info_gse70138['pert_id'],compounds) & np.in1d(sig_info_gse70138['cell_id'],cell_lines)    
sig_info_gse70138=sig_info_gse70138[fil]
print('Shape GSE92742 sig_info:',sig_info_gse92742.shape)
print('Shape GSE70138 sig_info:',sig_info_gse70138.shape)

Shape GSE92742 sig_info: (16010, 12)
Shape GSE70138 sig_info: (12693, 8)


In [62]:
#for matching we use log10 concentration, we start with gse92742
assert len(set(sig_info_gse92742['pert_dose_unit']))==1 #all doses are in uM
sig_info_gse92742['pert_dose']=sig_info_gse92742['pert_dose'].astype(float)
fil=sig_info_gse92742['pert_dose']!=0.0 #remove 0 concentration instances
sig_info_gse92742=sig_info_gse92742[fil]
sig_info_gse92742['log10_pert_dose']=np.log10(sig_info_gse92742['pert_dose'].astype(float))

In [63]:
#takes some time
sig_info_gse92742_nearest=sig_info_gse92742.apply(get_closest_cc_ctrp,axis=1)
sig_info_gse92742=pd.concat([sig_info_gse92742,sig_info_gse92742_nearest],1)
fil=~pd.isnull(sig_info_gse92742['cpd_avg_pv'])
sig_info_gse92742=sig_info_gse92742[fil]
sig_info_gse92742.to_csv('../results/CTRP/sig_info_gse92742_viab.csv')

In [64]:
#let's do this with gse70138, gse70138 does not have pert_dose and pert_dose unit
#only pert_idose, which is a string like '10.0 um'
sig_info_gse70138['pert_dose']=sig_info_gse70138['pert_idose'].apply(lambda x:float(x.split()[0]))
sig_info_gse70138['pert_dose_unit']=sig_info_gse70138['pert_idose'].apply(lambda x:x.split()[1])
assert len(set(sig_info_gse70138['pert_dose_unit']))==1 #all doses are in um
#remove 0 and log transform
fil=sig_info_gse70138['pert_dose']!=0.0 #remove 0 concentration instances
sig_info_gse70138=sig_info_gse70138[fil]
sig_info_gse70138['log10_pert_dose']=np.log10(sig_info_gse70138['pert_dose'].astype(float))

In [65]:
#takes some time
sig_info_gse70138_nearest=sig_info_gse70138.apply(get_closest_cc_ctrp,axis=1)
sig_info_gse70138=pd.concat([sig_info_gse70138,sig_info_gse70138_nearest],1)
fil=~pd.isnull(sig_info_gse70138['cpd_avg_pv'])
sig_info_gse70138=sig_info_gse70138[fil]
sig_info_gse70138.to_csv('../results/CTRP/sig_info_gse70138_viab.csv')

Now we can read the corresponding gene expression signatures. We will only work with the actual measured (landmark) genes. If you are interested for the infered genes also (bing), you can selecte the genes with *pr_is_bing* (marked with comment). 

In [18]:
#also takes some time
sig_info_gse92742=pd.read_table('../results/CTRP/sig_info_gse92742_viab.csv',
                                sep=',',header=0,index_col=[0])
sig_info_gse70138=pd.read_table('../results/CTRP/sig_info_gse70138_viab.csv',
                                sep=',',header=0,index_col=[0])
sig_ids_gse70138=list(sig_info_gse70138['sig_id'])
sig_ids_gse92742=list(sig_info_gse92742['sig_id'])
gene_info=pd.read_table('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',sep='\t')
fil=gene_info['pr_is_lm']==1 # change the columns name to pr_is_bing if you are interested for all genes
gene_ids = list(gene_info.loc[gene_info.index[fil],'pr_gene_id'].astype(str))
signatures_gse92742=parse('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx', 
                          cid=sig_ids_gse92742,rid=gene_ids)
signatures_gse92742=signatures_gse92742.data_df.T
signatures_gse70138=parse('../data/LINCS/GSE70138/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                          cid=sig_ids_gse70138,rid=gene_ids)
signatures_gse70138=signatures_gse70138.data_df.T
signatures_gse70138.to_csv('../results/CTRP/signatures_gse70138_lm.csv',sep=',')
signatures_gse92742.to_csv('../results/CTRP/signatures_gse92742_lm.csv',sep=',')

Finally, we can merge our data regarding GSE70138 and GSE92742, and we are ready with the CTRP preprocessing step.

In [91]:
#merge sig info files
sig_info_gse92742=pd.read_table('../results/CTRP/sig_info_gse92742_viab.csv',
                                sep=',',header=0,index_col=[0])
sig_info_gse70138=pd.read_table('../results/CTRP/sig_info_gse70138_viab.csv',
                                sep=',',header=0,index_col=[0])

sig_info_gse92742.index=sig_info_gse92742['sig_id']
sig_info_gse92742=sig_info_gse92742.loc[:,['pert_id','cell_id','pert_iname',
                                            'log10_pert_dose','pert_itime',
                                           'log10_cpd_conc_umol','cpd_avg_pv']]
fil=np.abs(sig_info_gse92742['log10_pert_dose']-sig_info_gse92742['log10_cpd_conc_umol'])<0.2
sig_info_gse92742=sig_info_gse92742[fil]
    
sig_info_gse70138.index=sig_info_gse70138['sig_id']
sig_info_gse70138=sig_info_gse70138.loc[:,['pert_id','cell_id','pert_iname',
                                           'log10_pert_dose','pert_itime',
                                           'log10_cpd_conc_umol','cpd_avg_pv']]
fil=np.abs(sig_info_gse70138['log10_pert_dose']-sig_info_gse70138['log10_cpd_conc_umol'])<0.2
sig_info_gse70138=sig_info_gse70138[fil]

sig_info_l1000_ctrp=pd.concat([sig_info_gse92742,sig_info_gse70138],0)
sig_info_l1000_ctrp.to_csv('../results/CTRP/sig_info_ctrp_full.csv',sep=',')

In [59]:
sig_info_l1000_ctrp=pd.read_csv('../results/CTRP/sig_info_ctrp_full.csv',sep=',',header=0,index_col=0)

In [60]:
#merge signature files
signatures_gse92742=pd.read_table('../results/CTRP/signatures_gse92742_lm.csv',
                                  sep=',',header=0,index_col=[0])
signatures_gse70138=pd.read_table('../results/CTRP/signatures_gse70138_lm.csv',
                                  sep=',',header=0,index_col=[0])
#be sure that genes are in the same order in the two signature files
signatures_gse70138=signatures_gse70138.loc[:,signatures_gse92742.columns]

signatures_l1000_ctrp=pd.concat([signatures_gse92742,signatures_gse70138],0)
#be sure that experiments are in the same order than in the sig_info file
signatures_l1000_ctrp=signatures_l1000_ctrp.loc[sig_info_l1000_ctrp.index,:]

/Users/benceszalai/Library/Enthought/Canopy/edm/envs/DeepLearningStart/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  app.launch_new_instance()
/Users/benceszalai/Library/Enthought/Canopy/edm/envs/DeepLearningStart/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: read_table is deprecated, use read_csv instead.


In [61]:
# we will merge duplicate entires of signatures using the MODZ method 
# describen in the original CMAP/L1000 study
def calc_MODZ(data):
    """calculates MODZ based on the original CMAP/L1000 study
    use only lm genes for MODZ calculation! Uses LM_GENES global
    variable."""
    if len(data)==1:
        return data[LM_GENES].iloc[0]
    if len(data)==2:
        return np.mean(data[LM_GENES],0)
    else:
        CM=scor(data[LM_GENES].T)[0]
        fil=CM<0
        CM[fil]=0.01
        weights=np.sum(CM,1)-1
        weights=weights/np.sum(weights)
        weights=weights.reshape((-1,1))
        return pd.Series(np.dot(data[LM_GENES].T,weights).reshape((-1,1)[0]),index=data[LM_GENES].columns)

In [62]:
#we use only landmark genes for MODZ calculation
gene_info=pd.read_table('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',sep='\t')
fil=gene_info['pr_is_lm']==1
LM_GENES = list(gene_info.loc[gene_info.index[fil],'pr_gene_id'].astype(str))
signatures_l1000_ctrp[['pert_id','cell_id','log10_pert_dose','pert_itime']]=\
    sig_info_l1000_ctrp[['pert_id','cell_id','log10_pert_dose','pert_itime']]
signatures_l1000_ctrp=signatures_l1000_ctrp.groupby(['pert_id','cell_id',
                                                     'log10_pert_dose',
                                                     'pert_itime']).apply(calc_MODZ)

/Users/benceszalai/Library/Enthought/Canopy/edm/envs/DeepLearningStart/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  from ipykernel import kernelapp as app


In [22]:
sig_info_l1000_ctrp=sig_info_l1000_ctrp.drop_duplicates(['pert_id','cell_id','log10_pert_dose','pert_itime'])
sig_info_l1000_ctrp=sig_info_l1000_ctrp.sort_values(['pert_id','cell_id','log10_pert_dose','pert_itime'])
signatures_l1000_ctrp.index=sig_info_l1000_ctrp.index
signatures_l1000_ctrp.to_csv('../results/CTRP/signatures_merged_lm.csv',sep=',')
sig_info_l1000_ctrp.to_csv('../results/CTRP/sig_info_merged_lm.csv',sep=',')

We are done with CTRP preprocessing
## We can continoue with matching shRNA abundance data from Achilles
At first we will merge the two different versions of Achilles dataset we will use.


In [71]:
achilles204=pd.read_table('../data/Achilles/Achilles_v2.4.6.rnai.gct',
                           sep='\t',header=0,index_col=[0],skiprows=2)
achilles218=pd.read_table('../data/Achilles/achilles-v2-19-2-mapped-rnai_v1-data.gct',
                          sep='\t',header=0,index_col=[0],skiprows=2)
#remove tissue type from columns, keep only cell line
achilles204.columns=pd.Series(achilles204.columns).apply(lambda x: x.split('_')[0]).values
achilles218.columns=pd.Series(achilles218.columns).apply(lambda x: x.split('_')[0]).values
#remove Description columns
del achilles204['Description']
del achilles218['Description']
shRNAs=list(set(achilles204.index)|set(achilles218.index))
cells=list(set(achilles204.columns)|set(achilles218.columns))
achilles_merged=pd.DataFrame(index=shRNAs,columns=cells)
achilles_merged.loc[achilles204.index,achilles204.columns]=achilles204
achilles_merged.loc[achilles218.index,achilles218.columns]=achilles218
achilles_merged.to_csv('../results/Achilles/achilles_merged.csv',sep=',')

Now we select the corresponding cell lines / shRNAs from Achilles and LINCS-L1000.

In [3]:
achilles_merged=pd.read_table('../results/Achilles/achilles_merged.csv',
                              sep=',',header=0,index_col=[0])
sig_info = pd.read_csv('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_sig_info.txt',
                       low_memory=False,sep='\t',header=0,index_col=None)
fil=sig_info['pert_type']=='trt_sh'
sig_info=sig_info[fil]
#selecting common cell lines
cells=list(set(achilles_merged.columns)&set(sig_info['cell_id']))
achilles_merged=achilles_merged[cells]
fil=np.in1d(sig_info['cell_id'],cells)
sig_info=sig_info[fil]

And match instances between Achilles and LINCS-L1000.

In [4]:
#matching shRNAs between achilles and lincs-l1000
achilles_merged['Sequence']=pd.Series(achilles_merged.index).apply(lambda x:x.split('_')[0]).values
achilles_merged=achilles_merged.drop_duplicates('Sequence') #we drop same sequence shRNAs
sequence_anno55k=pd.read_table('../data/Achilles/CP0001_reference_20150109.csv',
                               sep=',',header=None,index_col=None)
sequence_anno55k.columns=['Barcode','ID','Target']
sequence_anno90k=pd.read_table('../data/Achilles/CP0003_reference_20150109.csv',
                               sep=',',header=None,index_col=None)
sequence_anno90k.columns=['Barcode','ID','Target']
sequence_anno=pd.concat([sequence_anno55k,sequence_anno90k])
sequence_anno.index=range(len(sequence_anno.index))
sequence_anno=sequence_anno.drop_duplicates('ID')
#achilles uses barcode
fil=np.in1d(achilles_merged['Sequence'],sequence_anno['Barcode'])
achilles_merged=achilles_merged[fil]
#lincs uses ID
fil=np.in1d(sig_info['pert_id'],sequence_anno['ID'])
sig_info=sig_info[fil]
#match achilles and LINCS
sequence_anno.index=sequence_anno['ID'].values
sig_info['Barcode']=sequence_anno.loc[sig_info['pert_id'].values,'Barcode'].values
sig_info['shRNA_abundance']=np.nan
achilles_merged=pd.melt(achilles_merged,id_vars=['Sequence'],
                        value_vars=achilles_merged.columns[:-1])
achilles_merged.columns=['Sequence','Cell_line','shRNA_abundance']
fil=~pd.isnull(achilles_merged['shRNA_abundance']) #remove shRNA - cell pairs with no measured viaiblity
achilles_merged=achilles_merged[fil]
sig_info['Barcode_Cell']=sig_info['Barcode']+'_'+sig_info['cell_id']
achilles_merged['Barcode_Cell']=achilles_merged['Sequence']+'_'+achilles_merged['Cell_line']
#keep only shRNA - cell line pairs, which are present in both datasets
barcode_cells=list(set(achilles_merged['Barcode_Cell'])&set(sig_info['Barcode_Cell']))
fil=np.in1d(achilles_merged['Barcode_Cell'],barcode_cells)
achilles_merged=achilles_merged[fil]
fil=np.in1d(sig_info['Barcode_Cell'],barcode_cells)
sig_info=sig_info[fil]
#add shrna abundance to sig_info
achilles_merged.index=achilles_merged['Barcode_Cell']
sig_info['shRNA_abundance']=achilles_merged.loc[sig_info['Barcode_Cell'].values,
                                                'shRNA_abundance'].values

In [5]:
#read signautres
sig_ids=list(sig_info['sig_id'])
gene_info=pd.read_table('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',sep='\t')
fil=gene_info['pr_is_lm']==1 # change the columns name to pr_is_bing if you are interested for all genes
gene_ids = list(gene_info.loc[gene_info.index[fil],'pr_gene_id'].astype(str))
signatures=parse('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx', 
                 cid=sig_ids,rid=gene_ids)
signatures=signatures.data_df.T
sig_info.index=sig_info['sig_id'].values
sig_info=sig_info.loc[signatures.index]
sig_info.to_csv('../results/Achilles/sig_info_gse92742_lm.csv',sep=',')
signatures.to_csv('../results/Achilles/signatures_gse92742_lm.csv',sep=',')

In [109]:
sig_info=pd.read_table('../results/Achilles/sig_info_gse92742_lm.csv',sep=',',
                      header=0,index_col=[0])
signatures=pd.read_table('../results/Achilles/signatures_gse92742_lm.csv',sep=',',
                        header=0,index_col=[0])

/Users/benceszalai/Library/Enthought/Canopy/edm/envs/DeepLearningStart/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  from ipykernel import kernelapp as app
/Users/benceszalai/Library/Enthought/Canopy/edm/envs/DeepLearningStart/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: read_table is deprecated, use read_csv instead.


In [110]:
gene_info=pd.read_table('../data/LINCS/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',sep='\t')
fil=gene_info['pr_is_lm']==1
LM_GENES = list(gene_info.loc[gene_info.index[fil],'pr_gene_id'].astype(str))
signatures[['pert_id','cell_id','pert_itime']]=sig_info[['pert_id','cell_id','pert_itime']]
signatures=signatures.groupby(['pert_id','cell_id','pert_itime']).apply(calc_MODZ)

/Users/benceszalai/Library/Enthought/Canopy/edm/envs/DeepLearningStart/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  if __name__ == '__main__':


In [113]:
signatures.head()

,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
cid,,,,,,,,,,,,,,,,,,,,,
KDD003_A549_96H:TRCN0000000011:-666,0.837717,0.229930,0.810059,-0.444160,0.170612,0.567034,-0.160392,-1.220103,-0.286552,-0.002845,...,-0.214185,-0.016037,0.058926,-0.862933,1.710806,0.745072,0.106293,0.787529,1.449159,0.509378
KDD003_HCC515_96H:TRCN0000000011:-666,0.604000,-0.711950,2.608300,0.296400,-0.563700,-0.173700,1.188200,-0.824050,-0.810750,0.000000,...,0.520000,-0.121000,-0.823200,0.035450,0.628850,0.133400,0.251550,-0.308250,-0.090550,-0.072250
KDD003_HT29_96H:TRCN0000000011:-666,-0.164350,-0.788850,1.310200,0.596000,1.086500,2.156700,0.523000,-3.381600,-2.412700,0.000000,...,-0.185800,0.323900,3.857850,0.286200,-0.187350,-0.573700,1.481650,0.013750,-1.216650,0.704100
KDD003_MCF7_96H:TRCN0000000011:-666,0.655896,-0.370896,0.654760,0.467305,-0.795637,0.116167,1.083438,-0.403488,0.248501,0.210375,...,0.717850,0.752322,-1.017231,-0.909209,1.928901,0.813918,1.622391,1.430153,-0.123830,1.335271
KDD003_PC3_96H:TRCN0000000011:-666,0.899881,-0.552286,-0.295795,0.487287,-0.478602,-0.985538,0.582110,-0.282072,0.690723,1.867573,...,0.784905,0.252051,0.051223,-0.230011,0.494382,-1.002934,1.096575,-0.233879,1.086871,-1.142171


In [13]:
sig_info=sig_info.drop_duplicates(['pert_id','cell_id','pert_itime'])
sig_info=sig_info.sort_values(['pert_id','cell_id','pert_itime'])
signatures.index=sig_info.index
signatures.to_csv('../results/Achilles/signatures_merged_lm.csv',sep=',')
sig_info.to_csv('../results/Achilles/sig_info_merged_lm.csv',sep=',')

We are done with Achilles preprocessing!

Later we will need CTRP dose response curve data (AUC and EC50 values) from CTRP dataset, we will preprocess them now too.

In [5]:
# read raw CTRP cell viability data
ctrp_curves=pd.read_table('../data/CTRP/v20.data.curves_post_qc.txt',
                       sep='\t',header=0,index_col=None)
#read CTRP metadata
cell_info=pd.read_table('../data/CTRP/v20.meta.per_cell_line.txt',
                        sep='\t',header=0,index_col=None)
compound_info=pd.read_table('../data/CTRP/v20.meta.per_compound.txt',
                            sep='\t',header=0,index_col=None)
experiment_info=pd.read_table('../data/CTRP/v20.meta.per_experiment.txt',
                              sep='\t',header=0,index_col=None)

In [6]:
#using information from metadata we prepocess CTRP data
#to have usable format to match with LINCS
ctrp_proc=ctrp_curves.loc[:,['experiment_id','master_cpd_id','pred_pv_high_conc',
                          'apparent_ec50_umol','area_under_curve']].copy()
experiment_info=experiment_info.drop_duplicates(['experiment_id',
                                                 'master_ccl_id'])
experiment_info.index=experiment_info['experiment_id']
ctrp_proc['master_ccl_id']=experiment_info.loc[ctrp_proc['experiment_id'].values,
                                               'master_ccl_id'].values
cell_info.index=cell_info['master_ccl_id']
ctrp_proc['ccl_name']=cell_info.loc[ctrp_proc['master_ccl_id'].values,
                                    'ccl_name'].values
compound_info.index=compound_info['master_cpd_id']
ctrp_proc['broad_cpd_id']=compound_info.loc[ctrp_proc['master_cpd_id'].values,
                                            'broad_cpd_id'].values
ctrp_proc=ctrp_proc.loc[:,['ccl_name','broad_cpd_id','pred_pv_high_conc',
                           'apparent_ec50_umol','area_under_curve']]
ctrp_proc.to_csv('../results/CTRP/ctrp_DR.csv',sep=',')